# One-shot games

## Overview

- This is a compilation of results from several experiments where people play games like the [Prisoner's dilemma](https://en.wikipedia.org/wiki/Prisoner's_dilemma) or [Stag Hunt](https://en.wikipedia.org/wiki/Stag_hunt). See 'Games' for more information.

- This is also an attempt to replicate and work on [machine learning](https://en.wikipedia.org/wiki/Machine_learning) models that attempt to predict how people make decisions in these games. See 'ML models' for more information.

## Games

To contribute to this [database](https://github.com/polomsca/one-shot-games/blob/master/gamesmxn.csv), you can submit any experiment with two-player, one-shot, normal-form games in this format:

paper | game | matrixrow | matrixcol | choicerow | choicecol | shape | symmetric | n 
--- | --- | --- | --- | --- | --- | --- | --- | ---
stahlwilson1994 | 1 | 40 10 70; 20 80 0; 30 100 60 | | 11 40; 0 40; 29 40 | | 3 3 | 1 | 

- **paper** : Paper authors and date published. Add 'et al' after the first author for papers with more than two authors. See 'Papers' for more information.
- **game** : Game number if there are multiple games in one experiment. 
- **matrixrow** : Row player's payoff matrix. Separate elements with a space. Separate rows with a semicolon.
- **matrixcol** : May be blank if player results are pooled.
- **choicerow** : Row player's choice frequencies separated by semicolons. Fractions use a space instead of a slash. 
- **choicecol** : May be blank if player results are pooled.
- **shape** : Number of rows followed by number of columns. Separate values by a space.
- **symmetric** : '1' if symmetric, '0' otherwise.
- **n** : Number of subjects if **choicerow** is expressed in decimals or if number of subjects is otherwise unclear.

### Papers

- Haruvy et al (2001) : Modeling and testing for heterogeneity in observed strategic behavior

- Haruvy and Stahl (2007) : Equilibrium selection and bounded rationality in symmetric normal-form games

- Rogers et al (2009) : Heterogeneous quantal response equilibrium and cognitive hierarchies

- Stahl and Haruvy (2008) : Level-n bounded rationality and dominated strategies in normal-form games <sup>[1](#myfootnote1)</sup>

- Stahl and Wilson (1994) : Experimental evidence on players' models of other players

- Stahl and Wilson (1995) : On players' models of other players, theory and experimental evidence

- Goeree and Holt (2001) : Ten little treasures of game theory and ten intuitive contradictions 
    - (only games from "A Matching Pennies Game" and "The Kreps Game")

- Rydval and Ortmann (2005) : Loss avoidance as selection principle, evidence from simple stag-hunt games

- Haruvy and Stahl (1998) : An empirical model of equilibrium selection in symmetric normal-form games

- Costa-Gomes et al (1998) : Cognition and behavior in normal-form games, an experimental study 
    - (does not include TS treatment)

### Footnotes

[<a name="myfootnote1">1</a>] Appendix D, Game 10 is printed incorrectly as

12 | 63 | 22 
--- | --- | ---
0 | 0 | 38 
55 | 25 | 40 
35 | 35 | 43 

Game 10 should be read instead as

0 | 12 | 63 
--- | --- | ---
25 | 0 | 0
0 | 55 | 25 
100 | 35 | 35 

## ML models
The following is my attempt to replicate results from: 

- Hartford et al (2016) : Deep learning for predicting human strategic behavior
    - 'Gamenet'

## Session

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import string as string
from __future__ import division
sess = tf.InteractiveSession()

## Data

In [2]:
import_csv = pd.read_csv('gamesmxn.csv')

### Combine same games

In [ ]:
for i in range(1, import_csv.shape[0]):
    if pd.DataFrame(import_csv['matrixrow'][0:i] == import_csv['matrixrow'][i]).values.any():
        repeatmat = import_csv['matrixrow'][0:i][import_csv['matrixrow'][0:i] == import_csv['matrixrow'][i]].index[0]
        choicesum = np.matrix(import_csv['choicerow'][repeatmat]) + np.matrix(import_csv['choicerow'][i])
        choicesum = string.replace(str(choicesum),'[','')
        choicesum = string.replace(str(choicesum),']]','')
        choicesum = string.replace(str(choicesum),']\n',';')
        import_csv.set_value(repeatmat, 'choicerow', choicesum)
        import_csv.drop([i], inplace=True)

### Select games to use

In [3]:
tmp = np.empty((import_csv.shape[0]))
tmp[:] = np.NAN
for i in range(import_csv.shape[0]):
    if (import_csv['shape'][i] == '3 3' 
        and import_csv['symmetric'][i] == 1 
        and import_csv['paper'][i] != 'stahlwilson1995'):
        tmp[i] = i
index = tmp[~np.isnan(tmp)]

In [4]:
inputs_row = np.zeros((index.shape[0], 3, 3, 2))
inputs_col = np.zeros((index.shape[0], 3, 3, 2))
target_row = np.zeros((index.shape[0], 3, 1))
for j in range(index.shape[0]):
    Ur = np.matrix(import_csv['matrixrow'][int(index[j])])
    Ur = (Ur-np.mean(Ur))/np.std(Ur)
    Uc = np.transpose(Ur)
    ar = np.matrix(import_csv['choicerow'][int(index[j])])
    if ar.shape[1] == 2:
        ar = ar[:,0]/ar[:,1]
    inputs_row[j, :, :, 0] = Ur
    inputs_row[j, :, :, 1] = Uc
    inputs_col[j, :, :, 0] = Uc
    inputs_col[j, :, :, 1] = Ur
    target_row[j] = ar

### Training and test sets 

In [5]:
shuffle = np.random.permutation(range(inputs_row.shape[0]))
index_train = shuffle[0:inputs_row.shape[0]//5*4] 
index_tests = shuffle[inputs_row.shape[0]//5*4:inputs_row.shape[0]]
inputs_row_train = inputs_row[index_train]
inputs_col_train = inputs_col[index_train]
target_row_train = target_row[index_train]
inputs_row_tests = inputs_row[index_tests]
inputs_col_tests = inputs_col[index_tests]
target_row_tests = target_row[index_tests]

## Model

### Parameters

#### Weights and bias

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#### Convolution and pooling

In [7]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [8]:
def rw_pool(x,c):
    x_max = tf.reduce_max(x, axis=2)
    x_til = tf.tile(x_max,[1,3,1])
    x_sha = tf.reshape(x_til,[-1,3,3,c])
    return tf.transpose(x_sha, perm=[0,2,1,3])

def cw_pool(x,c):
    x_max = tf.reduce_max(x, axis=1)
    x_til = tf.tile(x_max,[1,3,1])
    return tf.reshape(x_til,[-1,3,3,c])

### Input and target

In [9]:
x_row = tf.placeholder(tf.float32, shape=[None, 3, 3, 2])
x_col = tf.placeholder(tf.float32, shape=[None, 3, 3, 2])
y_ = tf.placeholder(tf.float32, shape=[None, 3, 1])

### Hidden layer 1 (row player)

Tensorflow version `0.12.1` uses `tf.concat(axis, values, name='concat')`, not `tf.concat(values, axis, name='concat')`

In [11]:
x_pool1 = tf.concat([x_row, rw_pool(x_row, 2), cw_pool(x_row, 2)], 3)

In [12]:
W_conv1 = weight_variable([1, 1, 6, 50])
b_conv1 = bias_variable([50])
h_conv1 = tf.nn.relu(conv2d(x_pool1, W_conv1) + b_conv1)

### Hidden layer 2 (row player)

Tensorflow version `0.12.1` uses `tf.concat(axis, values, name='concat')`, not `tf.concat(values, axis, name='concat')`

In [13]:
x_pool2 = tf.concat([h_conv1, rw_pool(h_conv1, 50), cw_pool(h_conv1, 50)], 3)

In [14]:
W_conv2 = weight_variable([1, 1, 150, 50])
b_conv2 = bias_variable([50])
h_conv2 = tf.nn.relu(conv2d(x_pool2, W_conv2) + b_conv2)

In [15]:
keep_prob = tf.placeholder(tf.float32)
h_conv2_drop = tf.nn.dropout(h_conv2, keep_prob)

<div style="text-align: right"> <h3> Hidden layer 1 (col player) </h3> </div>

Tensorflow version `0.12.1` uses `tf.concat(axis, values, name='concat')`, not `tf.concat(values, axis, name='concat')`

In [16]:
x_pool1_col = tf.concat([x_col, rw_pool(x_col, 2), cw_pool(x_col, 2)], 3)

In [17]:
W_conv1_col = weight_variable([1, 1, 6, 50])
b_conv1_col = bias_variable([50])
h_conv1_col = tf.nn.relu(conv2d(x_pool1_col, W_conv1_col) + b_conv1_col)

<div style="text-align: right"> <h3> Hidden layer 2 (col player) </h3> </div>

In [18]:
x_pool2_col = tf.concat([h_conv1_col, rw_pool(h_conv1_col, 50), cw_pool(h_conv1_col, 50)], 3)

In [19]:
W_conv2_col = weight_variable([1, 1, 150, 50])
b_conv2_col = bias_variable([50])
h_conv2_col = tf.nn.relu(conv2d(x_pool2_col, W_conv2_col) + b_conv2_col)

In [20]:
keep_prob_col = tf.placeholder(tf.float32)
h_conv2_col_drop = tf.nn.dropout(h_conv2_col, keep_prob_col)

<div style="text-align: right"> <h3> Action response layer 0 (col player) </h3> </div>

In [21]:
W_rwsm0 = weight_variable([1, 1, 50, 50])
h_rdsm1 = conv2d(h_conv2_col_drop, W_rwsm0)

In [22]:
ar_rwsm0_col = tf.reduce_sum(h_conv2_col_drop, axis=2)
ar_sfmx0_col = tf.nn.softmax(ar_rwsm0_col, dim=1)
ar_sfmx0_col = tf.slice(ar_sfmx0_col, [0, 0, 0], [-1, 3, 1])

### Action response layer 1 (row player response to col player)

In [23]:
W_rdsm1 = weight_variable([1, 1, 50, 50])
h_rdsm1 = conv2d(h_conv2_drop, W_rdsm1)

In [24]:
ar_rdsm1 = tf.reduce_sum(h_rdsm1, axis=3)
ar_dtpt1 = tf.matmul(ar_rdsm1, ar_sfmx0_col)

In [25]:
y = tf.nn.softmax(ar_dtpt1, dim=1)

### Cost function

In [36]:
beta = 0.01
regularizer = (tf.nn.l2_loss(W_conv1) 
               + tf.nn.l2_loss(W_conv2) 
               + tf.nn.l2_loss(W_conv1_col) 
               + tf.nn.l2_loss(W_conv2_col)
               + tf.nn.l2_loss(W_rwsm0) 
               + tf.nn.l2_loss(W_rdsm1))
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + beta * regularizer, reduction_indices=[1]))
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

### Optimization

In [37]:
train_step = tf.train.AdamOptimizer(0.0002,0.9,0.999,1e-8).minimize(cross_entropy)

## Train

In [38]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [39]:
for i in range(5000):
    if i%5 == 0:
        shuffle = np.random.permutation(range(inputs_row_train.shape[0]))
    inputs_row_train_batch = inputs_row_train[shuffle[shuffle.shape[0]//5*(i%5):shuffle.shape[0]//5*(i%5+1)]]
    inputs_col_train_batch = inputs_col_train[shuffle[shuffle.shape[0]//5*(i%5):shuffle.shape[0]//5*(i%5+1)]]
    target_row_train_batch = target_row_train[shuffle[shuffle.shape[0]//5*(i%5):shuffle.shape[0]//5*(i%5+1)]]
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
                x_row: inputs_row_train_batch,
                x_col: inputs_col_train_batch,
                y_: target_row_train_batch, 
                keep_prob: 1.0,
                keep_prob_col: 1.0})
        train_NLL = cross_entropy.eval(feed_dict={
                x_row: inputs_row_train_batch, 
                x_col: inputs_col_train_batch,
                y_: target_row_train_batch,
                keep_prob: 1.0,
                keep_prob_col: 1.0})
        print("step %d, train accuracy %g, train NLL %g"%(i, train_accuracy, train_NLL))
    train_step.run(feed_dict={x_row: inputs_row_train_batch, 
                              x_col: inputs_col_train_batch, 
                              y_: target_row_train_batch, 
                              keep_prob: 0.8,
                              keep_prob_col: 0.8})

step 0, train accuracy 0, train NLL 1.19007
step 100, train accuracy 0.8, train NLL 0.731989
step 200, train accuracy 1, train NLL 0.71316
step 300, train accuracy 0.866667, train NLL 0.832534
step 400, train accuracy 0.866667, train NLL 0.795097
step 500, train accuracy 0.933333, train NLL 0.806196
step 600, train accuracy 1, train NLL 0.752479
step 700, train accuracy 0.933333, train NLL 0.778576
step 800, train accuracy 1, train NLL 0.755481
step 900, train accuracy 1, train NLL 0.750553
step 1000, train accuracy 0.933333, train NLL 0.734352
step 1100, train accuracy 1, train NLL 0.820847
step 1200, train accuracy 0.933333, train NLL 0.733628
step 1300, train accuracy 1, train NLL 0.754738
step 1400, train accuracy 1, train NLL 0.670996
step 1500, train accuracy 0.933333, train NLL 0.684518
step 1600, train accuracy 0.933333, train NLL 0.788794
step 1700, train accuracy 1, train NLL 0.681856
step 1800, train accuracy 1, train NLL 0.73045
step 1900, train accuracy 1, train NLL 0.7547

## Test

In [40]:
test_accuracy = accuracy.eval(feed_dict={
        x_row: inputs_row_tests,
        x_col: inputs_col_tests,
        y_: target_row_tests, 
        keep_prob: 1.0,
        keep_prob_col: 1.0})
test_NLL = cross_entropy.eval(feed_dict={
        x_row: inputs_row_tests,
        x_col: inputs_col_tests,
        y_: target_row_tests, 
        keep_prob: 1.0,
        keep_prob_col: 1.0})
print("test accuracy %g, test NLL %g"%(test_accuracy, test_NLL))

test accuracy 1, test NLL 0.740187


In [43]:
pd.set_option('display.max_rows', None)
compare_y = y.eval(feed_dict={
                    x_row: inputs_row_tests, 
                    x_col: inputs_col_tests, 
                    y_: target_row_tests, 
                    keep_prob: 1.0, 
                    keep_prob_col: 1.0})
compare_y = compare_y.reshape(compare_y.shape[0], compare_y.shape[1])
compare_y_ = y_.eval(feed_dict={
                        x_row: inputs_row_tests,
                        x_col: inputs_col_tests,
                        y_: target_row_tests, 
                        keep_prob: 1.0,
                        keep_prob_col: 1.0})
compare_y_ = compare_y_.reshape(compare_y_.shape[0], compare_y_.shape[1])
compare = pd.DataFrame(np.concatenate((compare_y, compare_y_), axis=1))
compare.columns = ["y1","y2","y3","y_1","y_2","y_3"]
compare['RMSE 1'] = np.sqrt((compare['y1'] - compare['y_1'])**2)
compare['RMSE 2'] = np.sqrt((compare['y2'] - compare['y_2'])**2)
compare['RMSE 3'] = np.sqrt((compare['y3'] - compare['y_3'])**2)
compare

,y1,y2,y3,y_1,y_2,y_3,RMSE 1,RMSE 2,RMSE 3
0,0.343860,0.626594,0.029545,0.350000,0.650000,0.000000,0.006140,0.023406,0.029545
1,0.288495,0.018925,0.692580,0.240000,0.170000,0.590000,0.048495,0.151075,0.102580
2,0.509489,0.213682,0.276829,0.440000,0.260000,0.300000,0.069489,0.046318,0.023171
3,0.039103,0.948503,0.012394,0.027211,0.945578,0.027211,0.011892,0.002925,0.014817
4,0.035590,0.047360,0.917050,0.020408,0.047619,0.931973,0.015181,0.000259,0.014922
5,0.005410,0.071876,0.922714,0.000000,0.160000,0.840000,0.005410,0.088124,0.082714
6,0.673894,0.079664,0.246441,0.693878,0.102041,0.204082,0.019983,0.022376,0.042360
7,0.805431,0.058349,0.136220,0.765957,0.021277,0.212766,0.039473,0.037073,0.076546
8,0.922429,0.008202,0.069369,0.829787,0.021277,0.148936,0.092642,0.013074,0.079568
9,0.611113,0.360435,0.028452,0.666667,0.312925,0.020408,0.055553,0.047510,0.008044
